In [1]:
import ee
import json
import pandas as pd
import os
import numpy as np
import fiona
import folium
import geopandas as gpd
import random
import asf_search as asf
from shapely.geometry import Polygon
from shapely.geometry import mapping

workspace_directory = r'C:\Users\trygg\Documents\Master_project' #HOME
# workspace_directory = r'C:\Users\tryggvisi\Documents\my-awesome-masters-project' #WORK
os.chdir(workspace_directory)# Change the current working directory to the specified workspace


In [2]:
#Vindefjallen data
#import the excel file with Vindefjallen corner coordinates and measurements.
csv_file_path_2023 = r"Data\Vindefjallen_data\Vindefjallen_cleaning\Vindefjallen_Granule_processing\vindefjallen_corner_coords_processing_2023_middletime.csv"
df_measurements = pd.read_csv(csv_file_path_2023) # Read the CSV file into a DataFrame
vindefjallen_triangleCorners_numpy_2023 = df_measurements.to_numpy()# Convert the DataFrame to a NumPy array for easier coding
df_measurements.head()

,ID,OG_ID,Triangle,Corner,TriangleCorner,date,SnowDepth,SnowDepth_Noplus,temperature,x_4326,y_4326,x_3006,y_3006,MiddleTime
0,0,1,AC-42,A,AC-42A,2017-02-21,50,50,-5.0,15.760981,65.908407,534661.4522,7309911.059,10:50:00
1,1,2,AC-38,A,AC-38A,2017-02-26,50,50,-10.0,16.028612,65.896595,546872.4418,7308768.327,08:55:00
2,2,3,AC-37,A,AC-37A,2017-02-26,45,45,-8.0,15.848811,65.938003,538617.1852,7313260.876,11:27:30
3,3,4,AC-31,A,AC-31A,2017-02-26,20,20,-8.0,15.595766,65.984498,527055.8412,7318310.670,13:52:30
4,4,5,AC-39,A,AC-39A,2017-03-04,60,60,-15.0,15.936407,65.881455,542696.2178,7307015.189,09:35:00


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Assuming df_measurements contains the data and has a 'date' column
# You can adjust this to match your actual DataFrame structure

# Assuming date_counts is already calculated
date_counts = df_measurements['date'].value_counts().sort_index()

# Create a DataFrame from date_counts
df = pd.DataFrame(date_counts)
df.reset_index(inplace=True)
df.columns = ['date', 'count']

# Convert 'date' column to datetime
df['date'] = pd.to_datetime(df['date'])

# Group by year and sum the counts
df['year'] = df['date'].dt.year
yearly_counts = df.groupby('year')['count'].sum()

# Plotting
plt.figure(figsize=(10,6))
plt.rcParams.update({'font.size': 16})
yearly_counts.plot(kind='bar', color='skyblue')
# plt.xlabel('Year')
plt.xlabel('')
plt.ylabel('Count')
plt.title('Measurements Performed per Year')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.grid(axis='y')  # Add gridlines for y-axis
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()
#increase size of font

#hide xlabel


### Pre processing data

Multiple steps of converting the excel file received with the Vindelfjallen snow depth measurements into a more usable format.

This include, adding year 2023 which was received later, finding the average time of measurement, converting sweref to 4326 and more

#### Bæta við tíma við measurements

Mig langar að hafa nákvæmann mælitíma fyrir hverja mælingu.

Hver mæling hefur upphaf og enda, sem ég geri ráð fyrir að sé tíminn sem það tók að mæla einn þríhyrning.

Ég tek mismun á byrjunar og upphafstíma, finn miðju, og bæti við upphafstíma til að fá miðju-tíma

In [ ]:
# pd.options.mode.chained_assignment = None

# excel_file_path = r"Data\Vindefjallen_data\Vindefjallen_cleaning\Snowdepth_2017_2023_Editedby_MS_23.01.2024-copy.xlsx"
# sheet_name = "Census_data"
# df_census = pd.read_excel(excel_file_path, sheet_name = sheet_name)
# df_census_new =df_census[["ID", "Triangle","CensusDate (m/dd/yyyy)", "StartTime", "EndTime"]]
# print(df_census_new.head())
# df_census_new["StartTime"] = pd.to_datetime(df_census_new["StartTime"]).dt.time;
# df_census_new["EndTime"] = pd.to_datetime(df_census_new["EndTime"]).dt.time;
# #remove first row

# df_census_new.head()

# #Find the difference between the start and end time
# df_census_new["StartTime"] = pd.to_datetime(df_census_new["StartTime"], format='%H:%M:%S')
# df_census_new["EndTime"] = pd.to_datetime(df_census_new["EndTime"], format='%H:%M:%S')
# df_census_new["TimeDifference"] = df_census_new["EndTime"] - df_census_new["StartTime"]
# df_census_new["TimeDifference"] = df_census_new["TimeDifference"].dt.total_seconds()
# df_census_new["TimeDifference"] = df_census_new["TimeDifference"]/60
# df_census_new
# #Add time difference / 2 to the start time to get the middle time
# df_census_new["MiddleTime"] = df_census_new["StartTime"] + pd.to_timedelta(df_census_new["TimeDifference"]/2, unit='m')
# df_census_new["MiddleTime"] = df_census_new["MiddleTime"].dt.time
# df_census_new["MiddleTime"] = pd.to_datetime(df_census_new["MiddleTime"], format='%H:%M:%S')
# df_census_new["MiddleTime"] = df_census_new["MiddleTime"].dt.time
# df_census_new.head()
# df_census_new.dtypes

# #read csv 
# csv_file_path_2023 = r"Data\Vindefjallen_data\Vindefjallen_cleaning\Vindefjallen_Granule_processing\vindefjallen_corner_coords_processing_2023.csv"
# df_corners = pd.read_csv(csv_file_path_2023) # Read the CSV file into a DataFrame
# df_corners.head()

# df_corners['date'] = pd.to_datetime(df_corners['date'])

# #where df_corners Triangle and date matches df_census_new Triangle and CensusDate (m/dd/yyyy), add MiddleTime to df_corners
# df_corners["MiddleTime"] = np.nan
# for index, row in df_census_new.iterrows():
#     for index2, row2 in df_corners.iterrows():
#         if row["Triangle"] == row2["Triangle"] and row["CensusDate (m/dd/yyyy)"] == row2["date"]:
#             df_corners.at[index2, "MiddleTime"] = row["MiddleTime"]
# df_corners.head()


# df_corners
# #export to csv
# # df_corners.to_csv(r"Data\Vindefjallen_data\Vindefjallen_cleaning\Vindefjallen_Granule_processing\vindefjallen_corner_coords_processing_2023_middletime.csv", index = False)

#### Import Vindefjallen TriangleCorner data into a numpy and pandas dataframes

Það er excel skrá með lista yfir hverja einustu Vindefjallen mælingu, þar sem hver
mæling hefur sína línu

In [ ]:
# #import the excel file with Vindefjallen corner coordinates and measurements.

# # csv_file_path = r"C:\Users\trygg\Documents\Master_project\Data\Vindefjallen_data\Vindefjallen_cleaning\vindefjallen_corner_coords_2023.csv"
# csv_file_path = r"Data\Vindefjallen_data\Vindefjallen_cleaning\vindefjallen_corner_coords.csv" #The file is a csv file where each single measurement is a row.
# df_measurements = pd.read_csv(csv_file_path) # Read the CSV file into a DataFrame
# df_measurements_filter = df_measurements.dropna(subset=['SnowDepth']) #Remove all nan values from the dataframe, there are null values in the SnowDepth column in the excel file


# vindefjallen_triangleCorners_numpy = df_measurements_filter.to_numpy()# Convert the DataFrame to a NumPy array for easier coding
# df_measurements_filter.head()

#### Bæta við 2023 data

Byrjað á því að manually splitta original datasetinu yfir í A, B og C measurements. Síðan hreina það örlítið. 

Síðan er það að bæta við hnitum, bæði sweref og 4326

In [ ]:
# #Hérna er ég að búa til dataframe með unique TriangleCorner og coordinates

# csv_file_path_2023 = r"Data\Vindefjallen_data\Vindefjallen_cleaning\Vindefjallen_Granule_processing\vindefjallen_corner_coords_processing.csv"
# # Read the CSV file into a DataFrame
# df_measurements_filter_2023 = pd.read_csv(csv_file_path_2023)
# #Remove all nan values from the dataframe, there are null values in the SnowDepth column in the excel file
# df_measurements_filter_2023 = df_measurements_filter_2023.dropna(subset=['SnowDepth'])
# # Convert the DataFrame to a NumPy array for easier coding
# # vindefjallen_triangleCorners_numpy = df_measurements_filter.to_numpy()

# #Create a df  from df_measurements_filter with unique TriangleCorner along with the coordinates
# df_unique_corners = df_measurements_filter_2023.drop_duplicates(subset=['TriangleCorner'])
# df_unique_corners = df_unique_corners[['TriangleCorner', 'x_4326', 'y_4326', 'x_3006', 'y_3006']]
# df_unique_corners = df_unique_corners.reset_index(drop=True)
# # df_unique_corners

# #Hérna er verið að ná í öll gögnin úr excel skránni fyrir árið 2023
# excel_file_path = r"Data\Vindefjallen_data\Vindefjallen_cleaning\Snowdepth_2017_2023_Editedby_MS_23.01.2024-copy.xlsx"
# df_census = pd.read_excel(excel_file_path, sheet_name="Census_data_2023") # Read the CSV file into a DataFrame   
# df_census = df_census.dropna(subset=['SnowDepth']) #Remove all nan values from the dataframe, there are null values in the SnowDepth column in the excel file

# #Bæta við coords í census dataframeið
# df_census = df_census.merge(df_unique_corners, on='TriangleCorner', how='left')

# #export the 2023 dataframe to a csv file
# df_census.to_csv(r'Data\Vindefjallen_data\Vindefjallen_cleaning\vindefjallen_census_2023.csv', index=True)

#### Add temperature to each measurement

Þetta er botching vegna þess að ég bætti ekki við neinum temp tölum við vindefjallen_corner_coords. En þetta reddaðist vel með þessu script

Þetta virkar með því að importa temp gögnum úr sér file, og joina þau saman við vindefjallen numpy listann. En þetta eru líka á numpy formi þannig að síðistu línurnar eru til þess að sameina tvo numpy lista :)

!!! Þetta er gamalt, búið og gert!

In [ ]:
# csv_file_path = r"Data\Vindefjallen_data\Vindefjallen_cleaning\Triangle_temperature.csv"
# # Read the CSV file into a DataFrame
# df_temperature = pd.read_csv(csv_file_path)

# vindefjallen_temperature_numpy = df_temperature.to_numpy()

# temp_list = []
# for measurement in vindefjallen_triangleCorners_numpy:
#     for temperature in vindefjallen_temperature_numpy:
#         if temperature[0] == measurement[1] and temperature[1] == measurement[3]:
#             temp_list.append(temperature[2])
            
 
# my_array = np.array(temp_list).reshape(-1, 1) #Numpy behind the scenes
# # my_array = my_array

# vindefjallen_triangleCorners_numpy_celcius = np.hstack((vindefjallen_triangleCorners_numpy, my_array))
# vindefjallen_triangleCorners_numpy_celcius[0]

#### Add SWEREF coordinates back to dataset


!!! ÞEtta er gamalt, búið og gert

In [ ]:
# ##Adding sweref coords back to list with every single measurement (except null values)
# #also some modification of the table

# sweref_csv_file_path = r"Data\Vindefjallen_data\Vindefjallen_cleaning\triangleCorner_SWEREF.csv"
# vinde_measurements = r"Data\Vindefjallen_data\Vindefjallen_cleaning\vindefjallen_corner_coords_celcius.csv"

# df_SWEREF= pd.read_csv(sweref_csv_file_path)
# df_SWEREF.head(5)

# df_measurements = pd.read_csv(vinde_measurements)
# # df_measurements.head(5)
# df_SWEREF.head(5)
# #convert vinde_measurements to numpy array
# vinde_measurements_numpy = df_measurements.to_numpy()
# vindefjallen_SWEREF_numpy = df_SWEREF.to_numpy()



# temp_list_x = []
# temp_list_y = []
# for measurement in vinde_measurements_numpy:
#     for coords in vindefjallen_SWEREF_numpy:
#         # if temperature[0] == measurement[1] and temperature[1] == measurement[3]:
#             # temp_list.append(temperature[2])
        
#         if coords[0] == measurement[5]:
#             # temp_list.append(coords[2])
#             # print(coords)
#             temp_list_x.append(coords[1])
#             temp_list_y.append(coords[2])
            
# # print(temp_list_x)
# # print(temp_list_y)
# my_array_x = np.array(temp_list_x).reshape(-1, 1) #Numpy behind the scenes
# # my_array_x
# my_array_y = np.array(temp_list_y).reshape(-1, 1) #Numpy behind the scenes
# # my_array_y

# vindefjallen_corner_coords_celcius_sweref= np.hstack((vindefjallen_triangleCorners_numpy_celcius, my_array_x, my_array_y))

# vindefjallen_corner_coords_celcius_sweref

# df_vindefjallen_sweref = pd.DataFrame(vindefjallen_corner_coords_celcius_sweref, columns = ['OG_ID', 'Triangle', 'Corner', 'date', 'SnowDepth', 'TriangleCorner', 'x_4326', 'y_4326', 'temperature', 'x_3006', 'y_3006'])
# df_vindefjallen_sweref

# #rearange column order in df
# df_vindefjallen_sweref = df_vindefjallen_sweref[['OG_ID', 'Triangle', 'Corner', 'TriangleCorner', 'date', 'SnowDepth', 'temperature', 'x_4326', 'y_4326', 'x_3006', 'y_3006']]
# df_vindefjallen_sweref
# df_vindefjallen_sweref.to_csv(r'Data\Vindefjallen_data\Vindefjallen_cleaning\vindefjallen_corner_coords_processing.csv', index = True)

In [ ]:
# ### Add SWEREF COORDS to filtered measurements

# csv_file_path = r"Data\Vindefjallen_data\Vindefjallen_cleaning\triangleCorner_SWEREF.csv"
# # Read the CSV file into a DataFrame
# df_SWEREF= pd.read_csv(csv_file_path)
# df_SWEREF.head(5)
# vindefjallen_SWEREF_numpy = df_SWEREF.to_numpy()

# temp_list_x = []
# temp_list_y = []
# for measurement in vindefjallen_triangleCorners_numpy:
#     for coords in vindefjallen_SWEREF_numpy:
#         # if temperature[0] == measurement[1] and temperature[1] == measurement[3]:
#             # temp_list.append(temperature[2])
        
#         if coords[0] == measurement[5]:
#             # temp_list.append(coords[2])
#             # print(coords)
#             temp_list_x.append(coords[1])
#             temp_list_y.append(coords[2])
            
# print(temp_list_x)
# print(temp_list_y)
# my_array_x = np.array(temp_list_x).reshape(-1, 1) #Numpy behind the scenes
# my_array_x
# my_array_y = np.array(temp_list_y).reshape(-1, 1) #Numpy behind the scenes
# my_array_y

# vindefjallen_triangleCorners_numpy_celcius_sweref = np.hstack((vindefjallen_triangleCorners_numpy_celcius, my_array_x, my_array_y))

# vindefjallen_triangleCorners_numpy_celcius_sweref[1]


In [ ]:
# #convert to pandas dataframe

# 'ID', 'Triangle', 'Corner', 'date', 'SnowDepth', 'CornerTriangle', 'longitude', 'latitude', 'temperature'
# df_vindefjallen_celcius = pd.DataFrame(vindefjallen_triangleCorners_numpy_celcius, columns = ['ID', 'Triangle', 'Corner', 'date', 'SnowDepth', 'CornerTriangle', 'longitude', 'latitude', 'temperature'])
# df_vindefjallen_celcius
# #export to xlsx
# df_vindefjallen_celcius.to_csv(r'Data\Vindefjallen_data\Vindefjallen_cleaning\vindefjallen_corner_coords_celcius.csv', index = False)

### Create the big dictionary

Create a dictionary where each unique granule has information about what measurements correspond to it's footprint and date are added as a as a line in the dictionary.

Flow:

Go through every ASF item which has been date filtered and grabs the granule footprint.

For each item, it checks the entire Vindefjallen data set to test if the XY coords are located within the granule footprint,
and add relevant columns to the dataset if conditions are true.

If there is a match, then it is added to the dictionary.

The end result is a dictinary with a list of every granule that has at least one measurement within the footprint and other relevant information regarding the measurements.

In [3]:
def Point_in_polygon(x, y, polygon):
    """Check if a point (x, y) is inside a polygon defined by a list of coordinates.
    Parameters:
    - x, y: Coordinates of the point to be checked.
    - polygon: List of tuples representing the vertices of the polygon.
    xxx ChatGPT"""
    n = len(polygon)
    inside = False

    p1x, p1y = polygon[0]
    for i in range(n + 1):
        p2x, p2y = polygon[i % n]
        if y > min(p1y, p2y) and y <= max(p1y, p2y) and x <= max(p1x, p2x):
            if p1y != p2y:
                xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                if p1x == p2x or x <= xinters:
                    inside = not inside
        p1x, p1y = p2x, p2y
    return inside

#### ASF

https://docs.asf.alaska.edu/asf_search/ASFSearchResults/

https://docs.asf.alaska.edu/asf_search/searching/

https://github.com/asfadmin/Discovery-asf_search/blob/405dcd6efe822ca08efdc24dd39d67849220101e/examples/1-Basic_Overview.ipynb

In [4]:
import asf_search as asf
#my_env home


# aoi = 'POLYGON((15.4889 66.3616,15.4752 66.354,15.4615 66.3464,15.4477 66.3389,15.434 66.3313,15.4203 66.3238,15.4066 66.3162,15.3929 66.3087,15.3792 66.3011,15.3656 66.2935,15.3519 66.286,15.3383 66.2784,15.3246 66.2709,15.311 66.2633,15.2973 66.2557,15.2837 66.2482,15.2701 66.2406,15.2565 66.233,15.2429 66.2254,15.2293 66.2179,15.2157 66.2103,15.2021 66.2027,15.1886 66.1951,15.175 66.1876,15.1615 66.18,15.1479 66.1724,15.1344 66.1648,15.1209 66.1572,15.1074 66.1496,15.0938 66.1421,15.0803 66.1345,15.0668 66.1269,15.0534 66.1193,15.0644 66.1151,15.0754 66.1109,15.0864 66.1067,15.0974 66.1025,15.1084 66.0983,15.1195 66.0941,15.1305 66.0899,15.1415 66.0857,15.1524 66.0815,15.1634 66.0773,15.1744 66.0731,15.1854 66.0689,15.1964 66.0647,15.2074 66.0605,15.2183 66.0563,15.2293 66.0521,15.2403 66.0479,15.2512 66.0437,15.2622 66.0395,15.2731 66.0353,15.2841 66.0311,15.295 66.0268,15.306 66.0226,15.3169 66.0184,15.3279 66.0142,15.3388 66.01,15.3497 66.0058,15.3607 66.0016,15.3716 65.9973,15.3825 65.9931,15.3934 65.9889,15.4043 65.9847,15.4152 65.9805,15.4261 65.9763,15.437 65.972,15.4479 65.9678,15.4588 65.9636,15.4697 65.9594,15.4806 65.9551,15.4915 65.9509,15.5024 65.9467,15.5132 65.9425,15.5241 65.9382,15.535 65.934,15.5458 65.9298,15.5567 65.9256,15.5676 65.9213,15.5784 65.9171,15.5893 65.9129,15.6001 65.9086,15.611 65.9044,15.6218 65.9002,15.6326 65.8959,15.6435 65.8917,15.6543 65.8875,15.6651 65.8832,15.6759 65.879,15.6868 65.8747,15.6976 65.8705,15.7084 65.8663,15.7192 65.862,15.73 65.8578,15.7408 65.8535,15.7516 65.8493,15.7684 65.8457,15.7852 65.842,15.8019 65.8384,15.8187 65.8348,15.8355 65.8311,15.8522 65.8275,15.869 65.8239,15.8857 65.8202,15.9025 65.8166,15.9192 65.813,15.9359 65.8093,15.9527 65.8057,15.9694 65.802,15.9861 65.7984,16.0028 65.7947,16.0195 65.7911,16.0388 65.7874,16.058 65.7836,16.0773 65.7799,16.0965 65.7762,16.1157 65.7724,16.1349 65.7687,16.1542 65.765,16.1734 65.7612,16.1926 65.7575,16.2118 65.7537,16.231 65.75,16.2502 65.7463,16.2694 65.7425,16.2885 65.7388,16.3077 65.735,16.3269 65.7312,16.346 65.7275,16.3652 65.7237,16.3843 65.7199,16.4035 65.7162,16.4226 65.7124,16.4418 65.7086,16.4609 65.7049,16.48 65.7011,16.4991 65.6973,16.5183 65.6935,16.5374 65.6897,16.5565 65.686,16.5756 65.6822,16.5946 65.6784,16.6137 65.6746,16.6328 65.6708,16.6487 65.68,16.6646 65.6892,16.6805 65.6985,16.6964 65.7077,16.7124 65.7169,16.7283 65.7261,16.7443 65.7353,16.7602 65.7445,16.7762 65.7537,16.7922 65.763,16.8002 65.7676,16.8082 65.7722,16.8162 65.7768,16.8242 65.7814,16.8322 65.786,16.8402 65.7906,16.8482 65.7952,16.8563 65.7998,16.8643 65.8044,16.8723 65.809,16.8803 65.8136,16.8884 65.8182,16.8964 65.8228,16.9044 65.8274,16.9125 65.832,16.9205 65.8366,16.9286 65.8412,16.9366 65.8458,16.9447 65.8504,16.9527 65.855,16.9608 65.8595,16.9688 65.8641,16.9769 65.8687,16.985 65.8733,16.993 65.8779,17.0011 65.8825,17.0092 65.8871,17.0172 65.8917,17.0253 65.8963,17.0334 65.9009,17.0415 65.9055,17.0496 65.9101,17.0577 65.9147,17.0658 65.9192,17.0739 65.9238,17.082 65.9284,17.0901 65.933,17.0982 65.9376,17.1063 65.9422,17.1144 65.9468,17.1225 65.9514,17.1306 65.9559,17.1387 65.9605,17.1469 65.9651,17.1214 65.9714,17.0958 65.9778,17.0703 65.9841,17.0448 65.9904,17.0192 65.9967,16.9936 66.003,16.9681 66.0093,16.9425 66.0156,16.9169 66.0219,16.8912 66.0282,16.8656 66.0345,16.84 66.0408,16.8143 66.0471,16.7886 66.0533,16.763 66.0596,16.7373 66.0659,16.7115 66.0721,16.6858 66.0784,16.6601 66.0846,16.6343 66.0909,16.6086 66.0971,16.5828 66.1034,16.557 66.1096,16.5312 66.1158,16.5054 66.1221,16.4796 66.1283,16.4538 66.1345,16.4279 66.1407,16.4021 66.1469,16.3762 66.1531,16.3503 66.1593,16.3244 66.1655,16.2985 66.1717,16.2726 66.1779,16.2466 66.1841,16.2207 66.1903,16.1947 66.1965,16.1687 66.2026,16.1427 66.2088,16.1167 66.215,16.0907 66.2211,16.0647 66.2273,16.0387 66.2334,16.0126 66.2396,15.9866 66.2457,15.9605 66.2519,15.9344 66.258,15.9083 66.2641,15.8822 66.2702,15.856 66.2764,15.8299 66.2825,15.8037 66.2886,15.7776 66.2947,15.7514 66.3008,15.7252 66.3069,15.699 66.313,15.6728 66.3191,15.6466 66.3251,15.6203 66.3312,15.5941 66.3373,15.5678 66.3434,15.5415 66.3494,15.5152 66.3555,15.4889 66.3616))'
aoi = 'POLYGON((15.5702 66.3423,15.4488 66.3354,15.4242 66.2903,15.346 66.2493,15.2874 66.2273,15.2277 66.2165,15.145 66.1704,15.1619 66.0775,15.2024 66.0619,15.3474 66.0313,15.5304 65.9813,15.6281 65.9312,15.63 65.8965,15.741 65.8534,15.8627 65.8514,16.0288 65.8348,16.1754 65.8159,16.2049 65.8012,16.339 65.7877,16.4512 65.8283,16.2466 65.864,16.121 65.9472,16.1321 66.0205,16.2592 66.0296,16.4784 65.9773,16.6739 65.9642,16.8497 65.9721,16.9572 66.0108,16.7932 66.0502,16.5999 66.0381,16.4575 66.0817,16.2843 66.1399,16.2622 66.1561,15.9537 66.251,15.5702 66.3423))'
opts_slc = {
    'platform': asf.PLATFORM.SENTINEL1,
    'beamMode': asf.BEAMMODE.IW,
    'start': '2017-01-01T01:00:00Z',
    'end': '2023-06-11T23:59:59Z',
    'processingLevel' : asf.PRODUCT_TYPE.SLC #SLC datatype
}
asf_coords_slc = asf.geo_search(intersectsWith=aoi, **opts_slc)
print(f'{len(asf_coords_slc)} results found - SLC')

# opts_grd = {
#     'platform': asf.PLATFORM.SENTINEL1,
#     'beamMode': asf.BEAMMODE.IW,
#     'start': '2017-01-01T01:00:00Z',
#     'end': '2023-06-11T23:59:59Z',
#     'processingLevel' : asf.PRODUCT_TYPE.GRD_HD #GRD_HD datatype
# }
# asf_coords_grd = asf.geo_search(intersectsWith=aoi, **opts_grd)

# print(f'{len(asf_coords_grd)} results found - GRD_HD')

3528 results found - SLC


##### ASF SLC dictionary

In [5]:
#Updated with 2023
ordabok_granule_id_slc = {}
teljari = 0
for asf_coord in asf_coords_slc:
    polygon = asf_coord.geometry['coordinates'][0]
    measurment_in_granule_ID = []
    measurment_in_granule_cornerTriangle = []
    measurment_in_granule_date = []
    measurment_in_granule_depth = []
    measurment_in_granule_temperature = []

    # for corner in vindefjallen_triangleCorners_numpy_celcius: #Old
    for corner in vindefjallen_triangleCorners_numpy_2023:
        point = (corner[9], corner[10])  # x, y coordinates from the excel file
        start_time_match = corner[5] == asf_coord.properties['startTime'][:10]

        if Point_in_polygon(point[0], point[1], polygon) and start_time_match:
            measurment_in_granule_ID.append(corner[0])
            measurment_in_granule_date.append(corner[5])
            measurment_in_granule_depth.append(corner[6])
            measurment_in_granule_cornerTriangle.append(corner[4])
            measurment_in_granule_temperature.append(corner[8])

    if measurment_in_granule_date:
        ordabok_granule_id_slc[teljari] = {
            'granule_ID': asf_coord.properties['sceneName'],
            'orbitProp': asf_coord.properties['flightDirection'],
            'pathNr': asf_coord.properties['pathNumber'],
            'frameNr': asf_coord.properties['frameNumber'],
            'date': asf_coord.properties['startTime'][:10],
            'startTime': asf_coord.properties['startTime'],
            'measurement_date': measurment_in_granule_date[0],
            'measurement_id': measurment_in_granule_ID,
            'measurement_cornerTriangle': measurment_in_granule_cornerTriangle,
            'measurement_depth': measurment_in_granule_depth,
            'measurement_temperature': measurment_in_granule_temperature,
            'nr_of_measurements': len(measurment_in_granule_ID),
            'polygon': polygon
        }
        teljari += 1

print('length:', len(ordabok_granule_id_slc))

length: 57


In [7]:
ordabok_granule_id_slc

{0: {'granule_ID': 'S1A_IW_SLC__1SDV_20230326T053811_20230326T053838_047811_05BE83_3E60',
  'orbitProp': 'DESCENDING',
  'pathNr': 139,
  'frameNr': 371,
  'date': '2023-03-26',
  'startTime': '2023-03-26T05:38:11.987Z',
  'measurement_date': '2023-03-26',
  'measurement_id': [429, 453, 476],
  'measurement_cornerTriangle': ['AC-28A', 'AC-28B', 'AC-28C'],
  'measurement_depth': ['130', '20', '45'],
  'measurement_temperature': [-8.0, -8.0, -8.0],
  'nr_of_measurements': 3,
  'polygon': [[15.808266, 65.282188],
   [16.830559, 66.867043],
   [10.90213, 67.362747],
   [10.239535, 65.759735],
   [15.808266, 65.282188]]},
 1: {'granule_ID': 'S1A_IW_SLC__1SDV_20230325T161511_20230325T161538_047803_05BE49_44F8',
  'orbitProp': 'ASCENDING',
  'pathNr': 131,
  'frameNr': 214,
  'date': '2023-03-25',
  'startTime': '2023-03-25T16:15:11.543Z',
  'measurement_date': '2023-03-25',
  'measurement_id': [432,
   433,
   437,
   439,
   456,
   457,
   461,
   463,
   479,
   480,
   484,
   486],
  'm

##### ASF SLC extra

In [6]:
for i in ordabok_granule_id_slc.items():
    print(i[1]['nr_of_measurements'], i[1]['date'])

3 2023-03-26
12 2023-03-25
9 2023-03-11
9 2023-03-06
6 2022-03-16
1 2022-03-14
15 2022-03-14
3 2022-03-09
3 2022-03-04
6 2021-05-03
6 2021-04-17
18 2021-04-12
6 2021-03-28
12 2021-03-26
2 2021-03-26
9 2021-03-18
12 2021-03-18
6 2021-03-16
3 2021-03-15
3 2021-03-15
4 2021-03-10
3 2020-05-01
6 2020-04-29
18 2020-04-29
15 2020-04-28
3 2020-04-21
6 2020-04-05
6 2020-03-19
3 2020-02-26
9 2019-04-08
3 2019-04-05
12 2019-03-26
3 2019-03-16
6 2019-03-15
9 2019-03-11
3 2019-03-05
12 2019-03-05
4 2018-04-02
6 2018-03-31
5 2018-03-30
7 2018-03-30
12 2018-03-30
12 2018-03-28
11 2018-03-23
12 2018-03-23
4 2018-03-15
8 2017-04-18
6 2017-04-16
3 2017-04-14
6 2017-03-31
2 2017-03-17
9 2017-03-06
3 2017-03-06
12 2017-03-04
9 2017-02-26
1 2017-02-21
1 2017-02-21


In [ ]:
#Export til excel
ordabok_granule_id_slc_df = pd.DataFrame.from_dict(ordabok_granule_id_slc, orient='index')
ordabok_granule_id_slc_df.head()

In [ ]:
ordabok_granule_id_slc_df.to_csv(r'Data\Vindefjallen_data\Vindefjallen_cleaning\vindefjallen_granule_id_slc2.csv', index = False)

##### ASF GRD dictionary

In [ ]:
#Nýtt frá chatgpt, cleaned code

ordabok_granule_id_grd = {}
teljari = 0

for asf_coord in asf_coords_grd:
    polygon = asf_coord.geometry['coordinates'][0]
    measurment_in_granule_ID = []
    measurment_in_granule_cornerTriangle = []
    measurment_in_granule_date = []
    measurment_in_granule_depth = []
    measurment_in_granule_temperature = []

    for corner in vindefjallen_triangleCorners_numpy_celcius:
        point = (corner[6], corner[7])  # x, y coordinates from the excel file
        if Point_in_polygon(point[0], point[1], polygon) and corner[3] == asf_coord.properties['startTime'][:10]:
            measurment_in_granule_cornerTriangle.append(corner[5])
            measurment_in_granule_date.append(corner[3])
            measurment_in_granule_depth.append(corner[4])
            measurment_in_granule_ID.append(corner[0])
            measurment_in_granule_temperature.append(corner[8])

    if measurment_in_granule_date:
        ordabok_granule_id_grd[teljari] = {
            'granule_ID': asf_coord.properties['sceneName'],
            'orbitProp': asf_coord.properties['flightDirection'],
            'pathNr': asf_coord.properties['pathNumber'],
            'frameNr': asf_coord.properties['frameNumber'],
            'date': asf_coord.properties['startTime'][:10],
            'startTime': asf_coord.properties['startTime'],
            'measurement_date': measurment_in_granule_date[0],
            'measurement_id': measurment_in_granule_ID,
            'measurement_cornerTriangle': measurment_in_granule_cornerTriangle,
            'measurement_depth': measurment_in_granule_depth,
            'measurement_temperature': measurment_in_granule_temperature,
            'nr_of_measurements': len(measurment_in_granule_ID),
            'polygon': polygon
        }
        teljari += 1

print('length:', len(ordabok_granule_id_grd))
ordabok_granule_id_grd

In [ ]:
#Updated with 2023
#GRD
ordabok_granule_id_grd = {}
teljari = 0

for asf_coord in asf_coords_grd:
    polygon = asf_coord.geometry['coordinates'][0]
    measurment_in_granule_ID = []
    measurment_in_granule_cornerTriangle = []
    measurment_in_granule_date = []
    measurment_in_granule_depth = []
    measurment_in_granule_temperature = []

    for corner in vindefjallen_triangleCorners_numpy_2023:
        point = (corner[9], corner[10])  # x, y coordinates from the excel file
        start_time_match = corner[5] == asf_coord.properties['startTime'][:10]

        if Point_in_polygon(point[0], point[1], polygon) and start_time_match:
            measurment_in_granule_ID.append(corner[0])
            measurment_in_granule_date.append(corner[5])
            measurment_in_granule_depth.append(corner[6])
            measurment_in_granule_cornerTriangle.append(corner[4])
            measurment_in_granule_temperature.append(corner[8])

    if measurment_in_granule_date:
        ordabok_granule_id_grd[teljari] = {
            'granule_ID': asf_coord.properties['sceneName'],
            'orbitProp': asf_coord.properties['flightDirection'],
            'pathNr': asf_coord.properties['pathNumber'],
            'frameNr': asf_coord.properties['frameNumber'],
            'date': asf_coord.properties['startTime'][:10],
            'startTime': asf_coord.properties['startTime'],
            'measurement_date': measurment_in_granule_date[0],
            'measurement_id': measurment_in_granule_ID,
            'measurement_cornerTriangle': measurment_in_granule_cornerTriangle,
            'measurement_depth': measurment_in_granule_depth,
            'measurement_temperature': measurment_in_granule_temperature,
            'nr_of_measurements': len(measurment_in_granule_ID),
            'polygon': polygon
        }
        teljari += 1

print('length:', len(ordabok_granule_id_grd))
ordabok_granule_id_grd

In [ ]:
ordabok_granule_id_grd

In [ ]:
for i in ordabok_granule_id_grd.items():
    print(i[1]['granule_ID'], i[1]['date'], i[1]['nr_of_measurements'], i[1]['orbitProp'])

In [ ]:
#count number of unique PathNr and frameNr are in ordabok_granule_id_grd
unique_pathNr = []
unique_frameNr = []
for i in ordabok_granule_id_grd.items():
    unique_pathNr.append(i[1]['pathNr'])
    unique_frameNr.append(i[1]['frameNr'])
print(len(set(unique_pathNr)))
print(len(set(unique_frameNr)))
#which is the most common pathNr and frameNr
from collections import Counter
print(Counter(unique_pathNr))
print(Counter(unique_frameNr))


In [ ]:
for i in ordabok_granule_id_grd.items():
    if i[1]['frameNr'] == 369:
        print('"' + i[1]['granule_ID']+'"'+ ',')

In [ ]:
for i in ordabok_granule_id_grd.items():
    print(i[1]['nr_of_measurements'], i[1]['date'])

In [ ]:
#export to excel
ordabok_granule_id_grd = pd.DataFrame.from_dict(ordabok_granule_id_grd, orient='index')
ordabok_granule_id_grd.head()

In [ ]:
ordabok_granule_id_grd.to_csv(r'Data\Vindefjallen_data\Vindefjallen_cleaning\vindefjallen_granule_id_grd.csv', index = False)

### Polygon shapefile maker

In [10]:
# Búa til polygon shapefile með öllum granule footprints
# Define the output shapefile path and schema
output_shapefile = "Data\Vindefjallen_data\outputs\shapefiles\polygon_granule_id_slc.shp"

# ordabok_granule_id_grd
# ordabok_granule_id_slc

ordabok_granule = ordabok_granule_id_slc

schema = {
    'geometry': 'Polygon',
    'properties': {
                'id': 'int',
                'granule_ID': 'str',
                'orbitProp': 'str',
                'pathNr': 'str',
                'frameNr': 'str',
                'date': 'str',
                'm_id' : 'str',
                'm_cT' : 'str',
                'm_depth' : 'str',
                'm_temp' : 'str',
                'nr_of_m' : 'str'
            }
        }
crs_wgs84 = {'init': 'epsg:4326'}  # 'epsg:4326' is the code for WGS84

# Create the shapefile
with fiona.open(output_shapefile, 'w', 'ESRI Shapefile', schema, crs=crs_wgs84) as output:
    # Create a polygon using the provided coordinates

    for i in range(len(ordabok_granule)): #looped through shapefile_listi which has a list of all identified granules
    # Define the properties for the polygon
        coords = [ordabok_granule[i]['polygon']]
        polygon = Polygon(coords[0])
        props = {
                'id': i,
                'granule_ID': ordabok_granule[i]['granule_ID'],
                'orbitProp': ordabok_granule[i]['orbitProp'],
                'pathNr': ordabok_granule[i]['pathNr'],
                'frameNr': ordabok_granule[i]['frameNr'],
                'date': ordabok_granule[i]['date'],
                'm_id' : ', '.join([str(s) for s in ordabok_granule[i]['measurement_id']]),
                'm_cT' : ', '.join([str(s) for s in ordabok_granule[i]['measurement_cornerTriangle']]),
                'm_depth' : ', '.join([str(s) for s in ordabok_granule[i]['measurement_depth']]),
                'm_temp' : ', '.join([str(s) for s in ordabok_granule[i]['measurement_temperature']]),
                'nr_of_m' : ordabok_granule[i]['nr_of_measurements']
            }

    # Write the feature to the shapefile
        output.write({
            'geometry': mapping(polygon),
            'properties': props,
        })

## Folium

In [ ]:
#Þessi kóði er til þess að búa til lista yfir hverja staka mælingu sem hefur granule
#Updated for 2023, go to git for old code
ordabok_granule_id = ordabok_granule_id_slc
def granule_every_point_dict_func(ordabok_granule_id):
    counter = 0
    granule_every_point_dict = {}
    for i in ordabok_granule_id:
        for j in range(len(ordabok_granule_id[i]['measurement_cornerTriangle'])):
            for k in vindefjallen_triangleCorners_numpy_2023:
                if ordabok_granule_id[i]['measurement_cornerTriangle'][j] == k[4] and ordabok_granule_id[i]['measurement_date'] == k[5]:
                    granule_every_point_dict[counter] = {
                        'granule_id': ordabok_granule_id[i]['granule_ID'],
                        'date': ordabok_granule_id[i]['measurement_date'],
                        'measurement_id': ordabok_granule_id[i]['measurement_id'][j],
                        'snow_depth': ordabok_granule_id[i]['measurement_depth'][j],
                        'temperature': ordabok_granule_id[i]['measurement_temperature'][j],
                        'corner': ordabok_granule_id[i]['measurement_cornerTriangle'][j],
                        'x_coord': k[9],
                        'y_coord': k[10],
                        'x_coord3006': k[11],
                        'y_coord3006': k[12]
                    }
                    counter += 1
    granule_df = pd.DataFrame.from_dict(granule_every_point_dict, orient='index')
    return granule_df

slc_granule_every_point_dict_df = granule_every_point_dict_func(ordabok_granule_id_slc)
# grd_granule_every_point_dict_df = granule_every_point_dict_func(ordabok_granule_id_grd)
slc_granule_every_point_dict_df.head(25)

In [ ]:
#Export slc_granule_every_point_dict_df to csv
slc_granule_every_point_dict_df.to_csv(r'Data\Vindefjallen_data\Vindefjallen_cleaning\slc_granule_every_point_2023.csv', index = False)

In [12]:
def flip_coords(footprint_str):
    coordinate_pairs = []
    for pair_str in footprint_str:
        temp = [round(float(pair_str[1]),2), round(float(pair_str[0]),2)]
        coordinate_pairs.append( temp)  # Swap lat and lon to follow (lon, lat) convention
    pairs_within_pairs = [coordinate_pairs]
    return pairs_within_pairs

In [13]:
#Ná í allar unique dagsetningar
def Unique_dates(dictionary):
    dates = []
    for i in range(len(dictionary)):
        dates.append(dictionary[i]['date'])
    unique_dates = list(dict.fromkeys(dates).keys())
    unique_dates.sort()
    return unique_dates

#### Búa til Folium kort

In [14]:
# shapefile_path = r'Data\Vindefjallen_data\outputs\shapefiles\points_with_granules1.shp' #old shp with missing points
# gdf = gpd.read_file(shapefile_path)


# shapefile_path = r'Data\Vindefjallen_data\outputs\shapefiles\points_with_granules_df_xy_export_2.shp' #New shp with all points (hopefully)
# gdf = gpd.read_file(shapefile_path)

path = r"Data\Vindefjallen_data\Vindefjallen_cleaning\slc_granule_every_point_2023.csv"
gdf = gpd.read_file(path)

vector_shp = r"Data\Vindefjallen_data\outputs\shapefiles\Vindefjallen_extent_updated4326.shp"
gdf_vector = gpd.read_file(vector_shp)
gdf.head()

,granule_id,date,measurement_id,snow_depth,temperature,corner,x_coord,y_coord,x_coord3006,y_coord3006,geometry
0,S1A_IW_SLC__1SDV_20230326T053811_20230326T0538...,2023-03-26,429,130,-8.0,AC-28A,15.8491718,66.15323252,538308.6585,7337249.602,None
1,S1A_IW_SLC__1SDV_20230326T053811_20230326T0538...,2023-03-26,453,20,-8.0,AC-28B,15.93648025,66.15941633,542236.8719,7337994.957,None
2,S1A_IW_SLC__1SDV_20230326T053811_20230326T0538...,2023-03-26,476,45,-8.0,AC-28C,15.90602943,66.12572896,540917.8704,7334220.113,None
3,S1A_IW_SLC__1SDV_20230325T161511_20230325T1615...,2023-03-25,432,80,-12.0,AC-34A,16.33018168,66.05549638,560237.5303,7326734.951,None
4,S1A_IW_SLC__1SDV_20230325T161511_20230325T1615...,2023-03-25,433,45,-10.0,AC-30A,16.32392487,66.07345377,559911.9498,7328730.193,None


In [15]:
# create a folium map
m = folium.Map(location=[66.0, 23.0], zoom_start=5)
feature_group = folium.FeatureGroup(
    name=f"Study area", show=False
)  # Create feature group which will be added to later
folium.GeoJson(gdf_vector).add_to(
    feature_group
)  # Add the vector file to the feature group
feature_group.add_to(m)
feature_group = folium.FeatureGroup(name=f"Vindefjällen measurement points", show=False)

In [16]:
for idx, row in gdf.iterrows():
    if row["corner"][-1:] == "A":
        folium.CircleMarker(
            location=[row['y_coord'], row['x_coord']],
            color="red",
            fill_color="red",
            radius=5,
            fill=True,
            fill_opacity=1,
            popup=folium.Popup(
                f"Corner: {row['corner']}<br>",
                max_width=500,
            ),
            tooltip=folium.Tooltip(text=f"{row['corner']}<br>"),
        ).add_to(feature_group)
        feature_group.add_to(m)

    elif row["corner"][-1:] == "B":
        folium.CircleMarker(
            location=[row['y_coord'], row['x_coord']],
            color="blue",
            fill_color="blue",
            radius=5,
            fill=True,
            fill_opacity=1,
            popup=folium.Popup(
                f"Corner: {row['corner']}<br>",
                max_width=500,
            ),
            tooltip=folium.Tooltip(text=f"{row['corner']}<br>"),
        ).add_to(feature_group)
        feature_group.add_to(m)
    elif row["corner"][-1:] == "C":
        # print(row["corner"][-1:])
        folium.CircleMarker(
            location=[row['y_coord'], row['x_coord']],
            color="green",
            fill_color="green",
            radius=5,
            fill=True,
            fill_opacity=1,
            popup=folium.Popup(
                f"Corner: {row['corner']}<br>",
                max_width=500,
            ),
            tooltip=folium.Tooltip(text=f"{row['corner']}<br>"),
        ).add_to(feature_group)
        feature_group.add_to(m)

In [17]:
#add only the first granule to the map
unique_dates = Unique_dates(ordabok_granule_id_slc)
for dagsetning in unique_dates:
    feature_group = folium.FeatureGroup(name=f"Group for {dagsetning}", show=True)
    for i in ordabok_granule_id_slc:
        random_color_poly = "#{:02x}{:02x}{:02x}".format(
            random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)
        )
        if ordabok_granule_id_slc[i]["date"] == dagsetning:
            polygon = flip_coords(ordabok_granule_id_slc[i]["polygon"])
            folium.Polygon(
                locations=polygon,
                color=random_color_poly,
                fill=True,
                fill_color=random_color_poly,
                fill_opacity=0.1,
                popup=folium.Popup(
                    f"GranuleID: {ordabok_granule_id_slc[i]['granule_ID']}<br>"
                    f"PathNr: {ordabok_granule_id_slc[i]['pathNr']}<br>"
                    f"FrameNr: {ordabok_granule_id_slc[i]['frameNr']}<br>"
                    f"Date: {ordabok_granule_id_slc[i]['date']}<br>"
                    f"Start Time: {ordabok_granule_id_slc[i]['startTime']}<br>"
                    f"Direction: {ordabok_granule_id_slc[i]['orbitProp']}",
                    max_width=500,
                ),
                tooltip=folium.Tooltip(
                    text=f"GranuleID: {ordabok_granule_id_slc[i]['granule_ID']}"
                ),
            ).add_to(feature_group)
            feature_group.add_to(m)
    for row in vindefjallen_triangleCorners_numpy_2023:
        if row[3] == dagsetning:
            folium.CircleMarker(
                location=[row[7], row[6]],
                color="black",
                fill_color="black",
                radius=3,
                fill=True,
                fill_opacity=1,
                popup=folium.Popup(
                    f"ID: {row[0]}<br>"
                    f"Corner: {row[5]}<br>"
                    f"Snow Depth: {row[4]} cm<br>"
                    f"Date: {row[3]}<br>",
                    max_width=500,
                ),
                tooltip=folium.Tooltip(text=f"Outside footprint<br>{row[5]}"),
            ).add_to(feature_group)
            feature_group.add_to(m)
    for idx, row in gdf.iterrows():
        if row["date"] == dagsetning:
            folium.CircleMarker(
                location=[row['y_coord'], row['x_coord']],
                color="red",
                fill_color="red",
                radius=5,
                fill=True,
                fill_opacity=1,
                popup=folium.Popup(
                    f"ID: {row['measurement_id']}<br>"
                    f"Corner: {row['corner']}<br>"
                    f"Snow Depth: {row['snow_depth']} cm<br>"
                    f"Date: {row['date']}<br>",
                    max_width=500,
                ),
                tooltip=folium.Tooltip(text=f"{row['corner']}<br>"),
            ).add_to(feature_group)
            feature_group.add_to(m)
    break
# unique_dates.pop(0)

In [18]:
#Add every granule and measurement to the map, and also the measurement points that did not correspond to a granule
unique_dates1 = Unique_dates(ordabok_granule_id_slc)
unique_dates = unique_dates1[1:]

for dagsetning in unique_dates:
    feature_group = folium.FeatureGroup(name=f"Group for {dagsetning}", show=False)

    for i in ordabok_granule_id_slc:
        if ordabok_granule_id_slc[i]["date"] == dagsetning:
            random_color_poly = "#{:02x}{:02x}{:02x}".format(
                random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)
            )
            polygon = flip_coords(ordabok_granule_id_slc[i]["polygon"])
            folium.Polygon(
                locations=polygon,
                color=random_color_poly,
                fill=True,
                fill_color=random_color_poly,
                fill_opacity=0.1,
                popup=folium.Popup(
                    f"GranuleID: {ordabok_granule_id_slc[i]['granule_ID']}<br>"
                    f"PathNr: {ordabok_granule_id_slc[i]['pathNr']}<br>"
                    f"FrameNr: {ordabok_granule_id_slc[i]['frameNr']}<br>"
                    f"Date: {ordabok_granule_id_slc[i]['date']}<br>"
                    f"Start Time: {ordabok_granule_id_slc[i]['startTime']}<br>"
                    f"Direction: {ordabok_granule_id_slc[i]['orbitProp']}",
                    max_width=500,
                ),
                tooltip=folium.Tooltip(
                    text=f"GranuleID: {ordabok_granule_id_slc[i]['granule_ID']}"
                ),
            ).add_to(feature_group)
            feature_group.add_to(m)
    for row in vindefjallen_triangleCorners_numpy_2023: #every measurement point, even if it did not correspond to a granule
        if row[3] == dagsetning:
            folium.CircleMarker(
                location=[row[7], row[6]],
                color="black",
                fill_color="black",
                radius=3,
                fill=True,
                fill_opacity=1,
                popup=folium.Popup(
                    f"ID: {row[0]}<br>"
                    f"Corner: {row[5]}<br>"
                    f"Snow Depth: {row[4]} cm<br>"
                    f"Date: {row[3]}<br>",
                    max_width=500,
                ),
                tooltip=folium.Tooltip(text=f"Outside footprint<br>{row[5]}"),
            ).add_to(feature_group)
            feature_group.add_to(m)
    for idx, row in gdf.iterrows(): #every measurement point that corresponded to a granule
        if row["date"] == dagsetning:
            folium.CircleMarker(
                location=[row['y_coord'], row['x_coord']],
                color="red",
                fill_color="red",
                radius=5,
                fill=True,
                fill_opacity=1,
                popup=folium.Popup(
                    f"ID: {row['measurement_id']}<br>"
                    f"Corner: {row['corner']}<br>"
                    f"Snow Depth: {row['snow_depth']} cm<br>"
                    f"Date: {row['date']}<br>",
                    max_width=500,
                ),
                tooltip=folium.Tooltip(text=f"{row['corner']}<br>"),
            ).add_to(feature_group)
            feature_group.add_to(m)

In [19]:
folium.LayerControl().add_to(m)
m.save('granules_and_measurentPoints_2023.html')
m

In [ ]:
m.save('granules_and_measurentPoints_2023.html')

### Debug



In [ ]:
#Ná í gögnin fyrir SLC granules
path_granuleID = r"C:\Users\tryggvisi\Documents\my-awesome-masters-project\Data\Vindefjallen_data\Vindefjallen_cleaning\vindefjallen_granule_id_slc.csv"
gdf2 = gpd.read_file(path_granuleID)
ordabok_granule_id_slc_excel = gdf2.to_dict('index')

In [ ]:
ordabok_granule_id_slc

In [ ]:
ordabok_granule_id_slc_excel

In [ ]:
print(len(ordabok_granule_id_slc_excel))
print(len(ordabok_granule_id_slc))


In [ ]:
path = r"C:\Users\tryggvisi\Documents\my-awesome-masters-project\Data\Vindefjallen_data\Vindefjallen_cleaning\slc_granule_every_point.csv"
gdf1 = gpd.read_file(path)
shapefile_path = r'Data\Vindefjallen_data\outputs\shapefiles\points_with_granules_df_xy_export_2.shp' #New shp with all points (hopefully)
gdf = gpd.read_file(shapefile_path)

print(len(gdf1))
print(len(gdf))

In [ ]:
# create a folium map
m = folium.Map(location=[66.0, 23.0], zoom_start=5)
feature_group = folium.FeatureGroup(
    name=f"Study area", show=False
)  # Create feature group which will be added to later
folium.GeoJson(gdf_vector).add_to(
    feature_group
)  # Add the vector file to the feature group
feature_group.add_to(m)

#Add every granule and measurement to the map, and also the measurement points that did not correspond to a granule
unique_dates1 = Unique_dates(ordabok_granule_id_slc)
unique_dates = unique_dates1

for dagsetning in unique_dates:
    feature_group = folium.FeatureGroup(name=f"Group for {dagsetning}", show=False)

    for i in ordabok_granule_id_slc:
        if ordabok_granule_id_slc[i]["date"] == dagsetning:
            random_color_poly = "#{:02x}{:02x}{:02x}".format(
                random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)
            )
            polygon = flip_coords(ordabok_granule_id_slc[i]["polygon"])
            folium.Polygon(
                locations=polygon,
                color=random_color_poly,
                fill=True,
                fill_color=random_color_poly,
                fill_opacity=0.1,
                popup=folium.Popup(
                    f"GranuleID: {ordabok_granule_id_slc[i]['granule_ID']}<br>"
                    f"PathNr: {ordabok_granule_id_slc[i]['pathNr']}<br>"
                    f"FrameNr: {ordabok_granule_id_slc[i]['frameNr']}<br>"
                    f"Date: {ordabok_granule_id_slc[i]['date']}<br>"
                    f"Start Time: {ordabok_granule_id_slc[i]['startTime']}<br>"
                    f"Direction: {ordabok_granule_id_slc[i]['orbitProp']}",
                    max_width=500,
                ),
                tooltip=folium.Tooltip(
                    text=f"GranuleID: {ordabok_granule_id_slc[i]['granule_ID']}"
                ),
            ).add_to(feature_group)
            feature_group.add_to(m)
    # for row in vindefjallen_triangleCorners_numpy_celcius: #every measurement point, even if it did not correspond to a granule
    #     if row[3] == dagsetning:
    #         folium.CircleMarker(
    #             location=[row[7], row[6]],
    #             color="black",
    #             fill_color="black",
    #             radius=3,
    #             fill=True,
    #             fill_opacity=1,
    #             popup=folium.Popup(
    #                 f"ID: {row[0]}<br>"
    #                 f"Corner: {row[5]}<br>"
    #                 f"Snow Depth: {row[4]} cm<br>"
    #                 f"Date: {row[3]}<br>",
    #                 max_width=500,
    #             ),
    #             tooltip=folium.Tooltip(text=f"Outside footprint<br>{row[5]}"),
    #         ).add_to(feature_group)
    #         feature_group.add_to(m)
    for idx, row in gdf.iterrows(): #every measurement point that corresponded to a granule
        if row["date"] == dagsetning:
            folium.CircleMarker(
                location=[row.geometry.y, row.geometry.x],
                color="red",
                fill_color="red",
                radius=5,
                fill=True,
                fill_opacity=1,
                popup=folium.Popup(
                    f"ID: {row['measuremen']}<br>"
                    f"Corner: {row['corner']}<br>"
                    f"Snow Depth: {row['snow_depth']} cm<br>"
                    f"Date: {row['date']}<br>",
                    max_width=500,
                ),
                tooltip=folium.Tooltip(text=f"{row['corner']}<br>"),
            ).add_to(feature_group)
            feature_group.add_to(m)
folium.LayerControl().add_to(m)

m

### Map GRD

#### Testa GRD og SLC

Vegna þess að GRD og SLC eru töluvert öðruvísi miðað við ASF

In [ ]:
# shapefile_path = r'Data\Vindefjallen_data\outputs\shapefiles\points_with_granules1.shp' #old shp with missing points
# gdf = gpd.read_file(shapefile_path)


# shapefile_path = r'Data\Vindefjallen_data\outputs\shapefiles\points_with_granules_df_xy_export_2.shp' #New shp with all points (hopefully)
# gdf = gpd.read_file(shapefile_path)

path = r"C:\Users\tryggvisi\Documents\my-awesome-masters-project\Data\Vindefjallen_data\Vindefjallen_cleaning\slc_granule_every_point.csv"
gdf = gpd.read_file(path)

vector_shp = r"Data\Vindefjallen_data\outputs\shapefiles\Vindefjallen_extent_updated4326.shp"
gdf_vector = gpd.read_file(vector_shp)






# create a folium map
m = folium.Map(location=[66.0, 23.0], zoom_start=5)
# feature_group = folium.FeatureGroup(
#     name=f"Study area", show=False
# )  # Create feature group which will be added to later
# folium.GeoJson(gdf_vector).add_to(
#     feature_group
# )  # Add the vector file to the feature group
# feature_group.add_to(m)
# feature_group = folium.FeatureGroup(name=f"Vindefjällen measurement points", show=False)

In [ ]:
m = folium.Map(location=[66.0, 23.0], zoom_start=5)

unique_dates = Unique_dates(ordabok_granule_id_slc)


for dagsetning in unique_dates:
    feature_group = folium.FeatureGroup(name=f"Group for {dagsetning}", show=False)

    for i in ordabok_granule_id_slc:
        if ordabok_granule_id_slc[i]["date"] == dagsetning:
            polygon = flip_coords(ordabok_granule_id_slc[i]["polygon"])
            folium.Polygon(
                locations=polygon,
                color='black',
                fill=True,
                fill_color='black',
                fill_opacity=0.1,
                popup=folium.Popup(
                    f"GranuleID: {ordabok_granule_id_slc[i]['granule_ID']}<br>"
                    f"PathNr: {ordabok_granule_id_slc[i]['pathNr']}<br>"
                    f"FrameNr: {ordabok_granule_id_slc[i]['frameNr']}<br>"
                    f"Date: {ordabok_granule_id_slc[i]['date']}<br>"
                    f"Start Time: {ordabok_granule_id_slc[i]['startTime']}<br>"
                    f"Direction: {ordabok_granule_id_slc[i]['orbitProp']}",
                    max_width=500,
                ),
                tooltip=folium.Tooltip(
                    text=f"GranuleID: {ordabok_granule_id_slc[i]['granule_ID']}"
                ),
            ).add_to(feature_group)
            feature_group.add_to(m)
    for idx, row in grd_granule_every_point_dict_df.iterrows(): #every measurement point that corresponded to a granule
        if row["date"] == dagsetning:
            folium.CircleMarker(
                location=[row['y_coord'], row['x_coord']],
                color="black",
                fill_color="black",
                radius=10,
                fill=True,
                fill_opacity=1,
                popup=folium.Popup(
                    f"ID: {row['measurement_id']}<br>"
                    f"Corner: {row['corner']}<br>"
                    f"Snow Depth: {row['snow_depth']} cm<br>"
                    f"Date: {row['date']}<br>",
                    max_width=500,
                ),
                tooltip=folium.Tooltip(text=f"{row['corner']}<br>"),
            ).add_to(feature_group)
            feature_group.add_to(m)
#---------------------------------------------#
            #GRD
    for i in ordabok_granule_id_grd:
        if ordabok_granule_id_grd[i]["date"] == dagsetning:
            polygon = flip_coords(ordabok_granule_id_grd[i]["polygon"])
            folium.Polygon(
                locations=polygon,
                color='red',
                fill=True,
                fill_color='red',
                fill_opacity=0.1,
                popup=folium.Popup(
                    f"GranuleID: {ordabok_granule_id_grd[i]['granule_ID']}<br>"
                    f"PathNr: {ordabok_granule_id_grd[i]['pathNr']}<br>"
                    f"FrameNr: {ordabok_granule_id_grd[i]['frameNr']}<br>"
                    f"Date: {ordabok_granule_id_grd[i]['date']}<br>"
                    f"Start Time: {ordabok_granule_id_grd[i]['startTime']}<br>"
                    f"Direction: {ordabok_granule_id_grd[i]['orbitProp']}",
                    max_width=500,
                ),
                tooltip=folium.Tooltip(
                    text=f"GranuleID: {ordabok_granule_id_grd[i]['granule_ID']}"
                ),
            ).add_to(feature_group)
            feature_group.add_to(m)

    for idx, row in slc_granule_every_point_dict_df.iterrows(): #every measurement point that corresponded to a granule
        if row["date"] == dagsetning:
            folium.CircleMarker(
                location=[row['y_coord'], row['x_coord']],
                color="red",
                fill_color="red",
                radius=5,
                fill=True,
                fill_opacity=1,
                popup=folium.Popup(
                    f"ID: {row['measurement_id']}<br>"
                    f"Corner: {row['corner']}<br>"
                    f"Snow Depth: {row['snow_depth']} cm<br>"
                    f"Date: {row['date']}<br>",
                    max_width=500,
                ),
                tooltip=folium.Tooltip(text=f"{row['corner']}<br>"),
            ).add_to(feature_group)
            feature_group.add_to(m)
folium.LayerControl().add_to(m)
m
#save map
m.save('slc_grd_granules_and_measurentPoints.html')

In [ ]:
for idx, row in slc_granule_every_point_dict_df.iterrows(): #every measurement point that corresponded to a granule
        if row["date"] == dagsetning:
            folium.CircleMarker(
                location=[row['y_coord'], row['x_coord']],
                color="red",
                fill_color="red",
                radius=5,
                fill=True,
                fill_opacity=1,
                popup=folium.Popup(
                    f"ID: {row['measurement_id']}<br>"
                    f"Corner: {row['corner']}<br>"
                    f"Snow Depth: {row['snow_depth']} cm<br>"
                    f"Date: {row['date']}<br>",
                    max_width=500,
                ),
                tooltip=folium.Tooltip(text=f"{row['corner']}<br>"),
            ).add_to(feature_group)
            feature_group.add_to(m)